Hello  everyone!  
This is a very quick tutorial on using transfer learning  to quickly use existing models and architecture for your own applications.  
We'll be using the ResNet network in this kernel, but similar if not identical methods works for other neural nets aswell.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import Model

from tqdm import tqdm
import cv2

from keras.applications.vgg19 import VGG19
from keras.layers import Dense, Flatten,Dropout
from keras.models import Sequential

from keras.layers import (AveragePooling2D, Convolution2D, Dense, Dropout, Flatten, Input, MaxPooling2D) 
from keras.models import Model, Sequential

print(os.listdir("../input/"))

After we checked out our input, we then proceed to read and save the data in a format that is more usefull to us.

In [ ]:
df_train = pd.read_csv('../input/dog-breed-identification/labels.csv')
train= ('../input/dog-breed-identification/train')
df_test = pd.read_csv('../input/dog-breed-identification/sample_submission.csv')
test= ('../input/dog-breed-identification/test')
resnet_weight_paths=("../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels.h5")

In [ ]:
df_test.head()

In [ ]:
df_train.head()

As we cab see the test and train datasets are very different in terms of content.  
If you're confused, the test dataset holds the possibilities of the dog belonging to that id.  
And the train set gives us the breed of each dog with their id attached. We'll be using these after a bit of formatting to train or neural net.

In [ ]:
len(df_train["breed"].unique())

Here I found how many different breeds of dogs we had in the dataset.  
We'll be using this later down the line to specify our top layer.

In [ ]:
num_classes = 120

Next we'll read and save all the images in the train set. We'll be using several loops to sort out the data in a meaningfull manner.  
We'll be resizing them to 90x90 images and turning them from images to pixel values out computer can actually use to train the model.

In [ ]:
target_labels = df_train["breed"]
one_hot = pd.get_dummies(target_labels, sparse = True)
one_hot_labels = np.asarray(one_hot)

What we did here is called "one hot encoding".  
This means is that we transformed the different labels into an array of 1s and 0s.  
This is done in a way such that every label is now held, but instead of by name, we store them by 1s and 0s.  
Here is a little demo of how we modified our dataset.

In [ ]:
demo = ["a","b","c","d","e"]
demo_df = pd.DataFrame(demo)
print(demo_df)
one_hot_demo = pd.get_dummies(demo_df,sparse = True)
one_hot_demo = np.asarray(one_hot_demo)
one_hot_demo

Here we have the loop to sort our data into two different arrays without changing the order.  
We use a number to keep track of the index to make sure our data doesn't get misallaigned.

In [ ]:
img_size = 90
x_train=[]
y_train=[]
i=0
for f in tqdm(df_train.id):
    img = cv2.imread('../input/dog-breed-identification/train/{0}.jpg'.format(f))
    label = one_hot_labels[i]
    x_train.append(cv2.resize(img, (img_size, img_size)))
    y_train.append(label)
    i += 1

Here the images are read, resized and saved into arrays.

In [ ]:
x_test = []
for f in tqdm(df_test.id):
    img = cv2.imread('../input/dog-breed-identification/test/{0}.jpg'.format(f))
    x_test.append(cv2.resize(img, (img_size,img_size))) 

All the pictures are then reformatted, and divided by 255. to reduce computational load.

In [ ]:
x_train_raw = np.array(x_train).astype("float32")/255.

In [ ]:
y_train_raw = np.array(y_train).astype("uint8")

In [ ]:
x_test = np.array(x_test, np.float32) / 255.

In [ ]:
print(x_train_raw.shape)
print(y_train_raw.shape)
print(x_test.shape)

Alright, now we are certain that our data is in the format we want it in, we can split this data into train and test datasets to validate our results.

In [ ]:
train_X,test_X,train_Y,test_Y = train_test_split(x_train_raw,y_train_raw,test_size = 0.3, random_state=3)

This is the first way of using transfer learning.  
With this method we copy the architecture of the VGG19 model to then train ourselves.

In [ ]:
base_model = VGG19(weights = None, include_top=False, input_shape=(img_size, img_size, 3))

# Add a new top layer
x = base_model.output
x = Flatten()(x)
predictions = Dense(num_classes, activation='softmax')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
for layer in base_model.layers:
    layer.trainable = False
    
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
#Take a look at the architecture.
callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
model.summary()

[Here](http://keras.io/layers/convolutional/) you can find what each of these layers actually do and how they interact. 

In [ ]:
model.fit(train_X, train_Y, epochs=1, validation_data=(test_X, test_Y), verbose=1)

In [ ]:
predictions = model.predict(x_test)

Now for the easier method, here we'll simply use the already existing weights trained on the official VGG19 network.

In [ ]:
weights = "../input/resnet50/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"
new_model = vgg19(weights = weights, include_top=False, input_shape=(img_size, img_size, 3))
new_model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
new_model.fit(train_X, train_Y, epochs=1, validation_data=(test_X, test_Y), verbose=1)
new_predicts = new_model.predict 

In [ ]:
#sub = pd.DataFrame(new_predict)
# Use the previous one_hot_encoded values to give the name to the columns
#col_names = one_hot.columns.values
#sub.columns = col_names

#sub.insert(0, 'id', df_test['id'])
#sub.head(5)